##### Copyright 2026 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Exploring Advanced Parameters with Updated Models

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/New_in_002.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook is updated for the `gemini-2.5 series` models that explores advanced parameters like `candidate_count`, `temperature`, `top_k`, and `top_p` for fine-grained output control. Unsupported parameters like `presence_penalty` and `frequency_penalty` are replaced by the model's inherent handling of repetition and diversity. It also demonstrates using response logprobs to analyze and refine outputs.

**Note**:

* This notebook was originally designed for `002` models. It has been updated to work with the latest models in the SDK, focusing on advanced parameter exploration.

## Setup

Install the latest version of the `google-genai` SDK:

In [ ]:
%pip install -U -q "google-genai>=1.0.0"

Import the package and set up your API key:

In [ ]:
from google import genai

In [ ]:
from google.colab import userdata
api_key = userdata.get("GEMINI_API_KEY")

In [ ]:
client = genai.Client(api_key=api_key)

Import other packages.

In [ ]:
from IPython.display import display, Markdown, HTML

Check model availibility:

In [ ]:
for model in client.models.list():
        print(model.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
mo

In [ ]:
model_name = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro"] {"allow-input":true, isTemplate: true}
test_prompt="Why don't people have tails?"

## Quick refresher on `generation_config` [Optional]

In [ ]:
from google.genai import types

In [ ]:
response = client.models.generate_content(
model= model_name,
contents='hello',
config=types.GenerateContentConfig(
temperature=1.0,
max_output_tokens=5,
),
)

Note:

* Each `generate_content` request is sent with a `generation_config` (`chat.send_message` uses `generate_content`).
* You can set the `generation_config` by either passing it to the model's initializer, or passing it in the arguments to `generate_content` (or `chat.send_message`).
* Any `generation_config` attributes set in `generate_content` override the attributes set on the model.
* You can pass the `generation_config` as either a Python `dict`, or a `genai.GenerationConfig`.
* If you're ever unsure about the parameters of `generation_config` check `genai.GenerationConfig`.

## Candidate count

you can also use `candidate_count > 1`.

In [ ]:
model = model_name

In [ ]:
generation_config = dict(candidate_count=2)

In [ ]:
response = client.models.generate_content(
    model= model,
    contents=test_prompt,
    config=generation_config
)

Here the `.text` quick-accessor returns the text result from the first candidate when multiple candidates are present.

In [ ]:
try:
  response.text # Does not fail with multiple candidates, Returns text from the first candidate when multiple candidates are present.
except ValueError as e:
  print(e)

When multiple candidates are present, iterate over `response.candidates` to access the content of each candidate individually:

In [ ]:
for candidate in response.candidates:
  display(Markdown(candidate.content.parts[0].text))
  display(Markdown("-------------"))


People don't have tails primarily because of **evolutionary adaptation to bipedalism (walking upright)**.

Here's a breakdown of the key reasons:

1.  **Bipedalism and Balance:**
    *   **Tails for Quadrupeds:** Many animals use their tails for balance when running, climbing, or leaping on four legs (quadrupeds). Think of a cheetah using its tail as a counterweight, or a monkey using its prehensile tail to grip branches.
    *   **Human Balance:** When early hominids began walking exclusively on two legs, their center of gravity shifted. Our pelvis, spine, and leg structure evolved to provide stability and balance for upright walking. A tail would actually be a hindrance, an unnecessary appendage that could throw off balance rather than aid it.

2.  **Dexterous Hands and Communication:**
    *   **Grasping:** Humans developed highly dexterous hands with opposable thumbs, which became incredibly efficient for grasping, manipulating tools, and climbing (when necessary). This eliminated the need for a prehensile (grasping) tail.
    *   **Communication:** Many animals use their tails for communication (e.g., dogs wagging, cats lashing, deer flagging). Humans, however, developed complex vocal language, intricate facial expressions, and sophisticated body language, making a communicative tail redundant.

3.  **Vestigial Remnant: The Coccyx (Tailbone):**
    *   While we don't have an external tail, we do have a **coccyx**, or tailbone, at the end of our spine. This is a vestigial structure, meaning it's a reduced and non-functional remnant of a feature that was present in our tailed ancestors. It's a clear piece of evidence that humans evolved from animals that *did* have tails.
    *   The genes for tail development are still present in our DNA, but they are typically "switched off" during human embryonic development.

4.  **No Functional Advantage:**
    *   In the course of human evolution, a tail simply ceased to offer any functional advantage and, in fact, became a potential liability (extra weight, potential for injury, no use for balance or manipulation). Natural selection favored individuals without tails as they were better adapted to their environment and lifestyle.

**In summary:** The absence of a tail in humans is a consequence of our unique evolutionary path, particularly our transition to upright walking and the development of sophisticated hands and communication methods.

-------------

Humans don't have tails because of a long evolutionary journey that led our ancestors to lose them. Here's a breakdown of the key reasons:

1.  **Evolutionary History and Common Ancestors:**
    *   **Shared Ancestry:** We share a common ancestor with many animals that *do* have tails, including monkeys.
    *   **The Divergence:** The lineage that led to humans, along with other great apes (chimpanzees, gorillas, orangutans, gibbons), diverged from the lineage that led to monkeys approximately 20-25 million years ago. It was around this time that the ancestors of apes lost their tails.

2.  **Adaptive Advantages (The "Why"):**
    *   **Locomotion Changes:**
        *   **Brachiation and Climbing:** Early apes developed different modes of locomotion. Instead of using tails for balance while running along branches (like many monkeys), they became adept at **brachiation** (swinging arm-over-arm through trees) or developed stronger grasping abilities for climbing. A tail might have been more of a hindrance than a help in these new movements.
        *   **Ground Dwelling:** As some ape lineages spent more time on the ground, the need for an arboreal balancing tail diminished further.
        *   **Bipedalism:** Much later, when our direct hominin ancestors began walking upright on two legs (**bipedalism**), a tail would have been actively disadvantageous, disrupting balance rather than aiding it, and requiring energy to maintain for no benefit.
    *   **Energy Efficiency:** Building and maintaining a tail (muscles, nerves, bone) requires energy. If a tail is no longer useful, natural selection would favor individuals who don't expend resources on it, allowing that energy to be used for other vital functions.

3.  **Genetic Basis (The "How"):**
    *   **Gene Mutation:** The loss of tails is attributed to a series of genetic mutations. Recent research, for example, points to a specific insertion in the *TBXT* gene (also known as *Brachyury*) that likely played a crucial role in suppressing tail development in early ape ancestors. This mutation essentially "turned off" the tail-growing program.
    *   **Developmental Pathways:** The genetic instructions for building a tail are still present in our DNA, but they are suppressed or modified during development.

4.  **Evidence of Our Tailless History:**
    *   **The Coccyx (Tailbone):** This small, fused set of vertebrae at the base of our spine is a vestigial structure – a remnant of a tail. It's the proof that our ancestors *did* have tails, and we still carry the basic anatomical blueprint.
    *   **Embryonic Tail:** Human embryos actually develop a tail-like structure for a few weeks during early gestation (around 4-6 weeks). This "embryonic tail" consists of 10-12 vertebrae and looks very much like a short tail. However, it is normally reabsorbed by the body, with the remaining vertebrae fusing to form the coccyx. This transient tail is a powerful piece of evidence for our evolutionary link to tailed ancestors.

In summary, humans lack tails because our ape ancestors, tens of millions of years ago, underwent genetic mutations that led to their loss. This loss was likely favored by natural selection because it offered advantages in their changing modes of locomotion and environments, eventually solidifying with the evolution of bipedalism in the human lineage.

-------------

The response contains multiple full `Candidate` objects.

In [ ]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""People don't have tails primarily because of **evolutionary adaptation to bipedalism (walking upright)**.

Here's a breakdown of the key reasons:

1.  **Bipedalism and Balance:**
    *   **Tails for Quadrupeds:** Many animals use their tails for balance when running, climbing, or leaping on four legs (quadrupeds). Think of a cheetah using its tail as a counterweight, or a monkey using its prehensile tail to grip branches.
    *   **Human Balance:** When early hominids began walking exclusively on two legs, their center of gravity shifted. Our pelvis, spine, and leg structure evolved to provide stability and balance for upright walking. A tail would actually be a hindrance, an unnecessary appendage that could throw off balance rather than aid it.

2.  **Dexterous Hands and Communication:**
    *   **Grasping:** Humans 

### Temperature, Top-k & Top-p Parameters

parameters like `temperature`, `top_k`, and `top_p` can be used to influence the diversity and randomness of the model's output.

* Temperature: Controls the randomness of token selection. Higher values (e.g., 1.5) increase diversity, while lower values (e.g., 0.2) make the output more deterministic.
* Top-k: Limits the sampling to the top k most probable tokens, reducing randomness and encouraging repetition.
* Top-p: Implements nucleus sampling, where the model considers the smallest set of tokens whose cumulative probability exceeds p.
These parameters provide fine-grained control over the token sampling process, allowing you to achieve effects similar to those previously achieved with the `presence_penalty` parameter.

In [ ]:
from statistics import mean

In [ ]:
def unique_words(prompt, generation_config, N=10):
  responses = []
  vocab_fractions = []
  for n in range(N):
    response = client.models.generate_content(
      model= model_name,
      contents=prompt,
      config=generation_config
    )
    responses.append(response)

    # Access the text content of the first candidate
    words = response.text.lower().split()
    score = len(set(words)) / len(words)
    print(score)
    vocab_fractions.append(score)

  return vocab_fractions

In [ ]:
prompt='Tell me a story'

In [ ]:
# baseline
v = unique_words(prompt, generation_config={})

0.5250338294993234
0.5386313465783664
0.5520974289580515
0.5456349206349206
0.5435393258426966
0.5512265512265512
0.5147727272727273
0.5535390199637024
0.5032679738562091
0.47992351816443596


In [ ]:
mean(v)

0.5307666641996984

In [ ]:
# these temperature, top_k, and top_p parameters can be used for diversity in output tokens.
v = unique_words(prompt, generation_config= dict(temperature=1.5))

0.5605700712589073
0.537746806039489
0.5265901981230449
0.5391304347826087
0.5413687436159347
0.5548098434004475
0.5195586760280843
0.5638888888888889
0.5224438902743143
0.588150289017341


NOTE:
* Penalty parameters (`presence_penalty`, `frequency_penalty`) are not supported in Gemini 2.5 models.


Migration Note:
* The penalty parameters previously available in older models (like `-002` versions) are no longer supported in the `Gemini 2.5` series. The new model architecture inherently handles issues like repetition, and attempts to set penalties will result in an `INVALID_ARGUMENT` error.

In [ ]:
mean(v)

0.545425784142906

In [ ]:
# parameters like temperature, top_k, top_p can also discourage diversity in the output tokens.
v = unique_words(prompt, generation_config=dict(temperature=0.2))

0.5230566534914362
0.4872665534804754
0.5427286356821589
0.5389221556886228
0.5248041775456919
0.5141579731743666
0.5514809590973202
0.5097783572359843
0.5432098765432098
0.5388127853881278


In [ ]:
mean(v)

0.5274218127327394

The `temperature`, `top_k`, and `top_p` parameters can be used to influence the diversity and randomness of the model's output. These parameters provide fine-grained control over the token sampling process, allowing you to achieve effects similar to those previously achieved with the `presence_penalty` parameter.

## Top-k Sampling

The `top_k` parameter limits the sampling to the top `k` most probable tokens, encouraging repetition and reducing randomness. This can be used as an alternative to the unsupported `frequency_penalty` parameter.

The easiest way to see that it works is to ask the model to do something repetitive. The model has to get creative while trying to complete the task.

In [ ]:
response = client.models.generate_content(
    model= model_name,
    contents='please repeat "Cat" 50 times, 10 per line, with random capitalization.',
    config=dict(top_k=10)
)

In [ ]:
print(response.candidates[0].content.parts[0].text)

Cat cAt CAT caT CAt Cat caT cat CaT CAT
cat CA T cAt Cat caT CAt CAT caT Cat CaT
CAT cAt Cat caT CAt cat CaT cAt CAT cat
CAt CaT cat CAT caT CAt cat CAT cAt Cat
Cat caT CAt cat CAT caT Cat cAt CaT CAT



> **Note**: The `frequency_penalty` parameter is not available in Gemini 2.5 models. Instead, the new model architecture inherently handles repetition issues. However, when the `max_output_tokens` limit is reached, the model may stop generating content prematurely, especially if internal reasoning consumes most of the tokens. To address this, consider adjusting parameters like `temperature`, `top_k`, and `top_p` to better control token usage and output diversity.

In [ ]:
from IPython.display import display, Markdown

In [ ]:
response = client.models.generate_content(
    model=model,
    contents= prompt,
    config=dict(
        max_output_tokens=400,
        top_p=0.2,
        temperature=0.2,
        )    # Controls randomness in token selection
)
print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=10>
) candidates=[Candidate(
  content=Content(
    role='model'
  ),
  finish_reason=<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='JgTxaP6pEMeHqtsPir-p0QY' usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=5,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=5
    ),
  ],
  thoughts_token_count=399,
  total_token_count=404
) automatic_function_calling_history=[] parsed=None


### Token Consumption in Gemini 2.5 Models

In the Gemini 2.5 models, the "thoughts" token count often consumes the majority of the allocated tokens, leaving little room for generating meaningful output. This behavior is particularly noticeable when using the `max_output_tokens` parameter, as shown in the example below.

Additionally, the `frequency_penalty` parameter, which was available in older models, is not supported in Gemini 2.5. The new model architecture inherently handles repetition issues, but this can lead to scenarios where token allocation is dominated by internal reasoning rather than output generation.

> **Observation**: The `finish_reason` in the response indicates `MAX_TOKENS`, highlighting that the model reached the token limit without producing meaningful content.

In [ ]:
Markdown(response.text)  # the, the, the, ...

<IPython.core.display.Markdown object>

In [ ]:
response.candidates[0].finish_reason

<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>